# Multi-layer LSTM for Music Generation (with Magenta)

In [17]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, BatchNormalization, Dropout, Activation
from magenta.scripts.convert_dir_to_note_sequences import convert_directory
from note_seq import music_pb2

small_file = 'small_sequences.tfrecord'
large_file = 'large_sequences.tfrecord'
note_divisions = 16

In [18]:
def create_network(network_input, n_vocab):
    """ Define constants """
    hidden_layers = 256
    dropout = 0.4
    temp = 0.6
    
    """ Initializing model """
    model = Sequential()
    
    """ Adding LSTM Layers to Model """
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True,
                input_shape=(network_input.shape[1], network_input.shape[2])
            )
        )
    )
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True
            )
        )
    )
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout
            )
        )
    )
    
    """ Add other layers after LSTM """
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(hidden_layers // 2))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(n_vocab))
    model.add(Lambda(lambda x: x / temp))
    model.add(Softmax())
    
    """ Define the optimizer and loss function for the model """
    model.compile(optimizer='adam', loss='categorical_crossentropy')
    
    return model
    

In [19]:
def convert_files():
    convert_directory(os.path.join(os.getcwd(), 'samples', 'small'), small_file, True)
    convert_directory(os.path.join(os.getcwd(), 'samples', 'large'), large_file, True)
convert_files()

INFO:tensorflow:Converting files in '/home/paulpan/GitRepos/final_project/samples/small/'.
INFO:tensorflow:0 files converted.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/small/086_DragonFighter_10_11Ending.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/small/087_DragonSpirit_TheNewLegend_17_18Ending.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/small/086_DragonFighter_00_01Title.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/small/087_DragonSpirit_TheNewLegend_00_01Start.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/small/087_DragonSpirit_TheNewLegend_16_17BossD.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/small/084_DragonBuster_09_10GameOver.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/small/086_DragonFighter_11_

INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/088_DragonWarrior_17_18CaveB4F.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/089_DragonWarriorII_04_05Deathfight.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/090_DragonWarriorIII_14_15Requiem.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/090_DragonWarriorIII_15_16Church.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/091_DragonWarriorIV_29_30Elegy.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/088_DragonWarrior_01_02People.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/089_DragonWarriorII_17_18UnknownWorld.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/090_DragonWarriorIII_22_23RainbowBri

INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/090_DragonWarriorIII_27_28MorninginAlefguard.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/090_DragonWarriorIII_26_27FightingSpirit.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/091_DragonWarriorIV_42_43IncarnationofEvil.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/088_DragonWarrior_21_22CaveB8F.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/088_DragonWarrior_14_15CaveB1F.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/091_DragonWarriorIV_17_18Save.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/090_DragonWarriorIII_12_13SmallShrine.mid.
INFO:tensorflow:Converted MIDI file /home/paulpan/GitRepos/final_project/samples/large/088_Dragon

In [20]:
# vocab key format "flag pitch"
# flag 0 is next channel, flag 1 is new note, flag 2 is next step, flag 3 is continue note

def is_start(target, start):
    return target + 0.02 > start and target - 0.02 < start

def is_note_valid(target, note, instrument_infos):
    return (
        target - 0.02 < note.end_time and
        target + 0.02 > note.start_time and
        instrument_infos[note.instrument].name != 'no'
    )

def get_vocab():
    reader = tf.data.TFRecordDataset(small_file)
    vocab = set(['0 0', '2 0'])
    for sequence in reader:
        data = music_pb2.NoteSequence.FromString(sequence.numpy())
        total_time = data.time_signatures[1].time
        iteration = 0
        while iteration / note_divisions < total_time:
            continue_notes = [
                note for note in data.notes
                if (not is_start(iteration / note_divisions, note.start_time) and
                is_note_valid(iteration / note_divisions, note, data.instrument_infos))
            ]
            new_notes = [note for note in data.notes if is_start(iteration / note_divisions, note.start_time)]
            for note in continue_notes:
                vocab.add(str(3) + ' ' + str(note.pitch))
            for note in new_notes:
                vocab.add(str(1) + ' ' + str(note.pitch))
            iteration += 1
    pickle.dump(vocab, open('vocab.p', 'wb'))
    return vocab

def get_notes():
    vocab = get_vocab()
    vocab_dict = {k: v for v, k in enumerate(vocab)}
    reader = tf.data.TFRecordDataset(small_file)
    notes = []
    for sequence in reader:
        data = music_pb2.NoteSequence.FromString(sequence.numpy())
        total_time = data.time_signatures[1].time
        iteration = 0
        while iteration / note_divisions < total_time:
            notes.append(vocab_dict['0 0'])
            all_notes = [
                note for note in data.notes
                if is_note_valid(iteration / note_divisions, note, data.instrument_infos)
            ]
            all_notes.sort(key=lambda note: note.instrument)
            prev_instrument = 0
            for note in all_notes:
                if note.instrument != prev_instrument:
                    if prev_instrument == 'p1':
                        if note.instrument == 'tr':
                            notes.append(vocab_dict['2 0'])
                        notes.append(vocab_dict['2 0'])
                    else:
                        notes.append(vocab_dict['2 0'])
                    prev_instrument = note.instrument
                if is_start(iteration / note_divisions, note.start_time):
                    notes.append(vocab_dict[str(1) + ' ' + str(note.pitch)])
                else:
                    notes.append(vocab_dict[str(3) + ' ' + str(note.pitch)])
            iteration += 1
    return notes

len(get_notes())

124714